In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score,roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
import optuna
import warnings
warnings.filterwarnings('ignore')

In [27]:
data = pd.read_csv('3months_under10_000.csv')
data

,shieldid,day,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,domain,created,updated,expires,whois
0,enilokmacy,2023-01-04,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
1,enilokmacy,2023-01-10,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
2,fecjavfec,2023-01-09,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
3,fecjavfec,2023-01-20,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
4,oreethecaw,2023-01-20,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,drift.com,1995-04-15,1995-04-15,2024-04-16,whois.ausregistry.net.au
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8267454,thwijijod,2022-12-21,cdn-prod-sg.yepgarage.info,yepgarage.info,1,20,2022-12-21 05:28:15.901,2022-12-21 05:28:15.901,0,1,0.0,yepgarage.info,2020-11-19,2020-11-19,2021-11-19,whois.gandi.net
8267455,thwijijod,2023-01-12,www.omaemirates.com,omaemirates.com,1,1,2023-01-12 06:01:10.576,2023-01-12 06:01:10.576,0,1,0.0,omaemirates.com,1998-10-28,1998-10-28,2020-10-27,whois.networksolutions.com
8267456,volcagbuca,2023-01-03,www.copenhagen.com,copenhagen.com,2,2,2023-01-03 01:18:13.956,2023-01-03 01:18:13.983,0,1,0.0,copenhagen.com,1996-04-02,1996-04-02,2026-04-02,whois.godaddy.com
8267457,volcagbuca,2023-01-20,www.jaiganticstudios.com,jaiganticstudios.com,1,1,2023-01-20 19:14:39.736,2023-01-20 19:14:39.736,0,1,0.0,jaiganticstudios.com,2017-05-01,2017-05-01,2021-05-01,whois.fastdomain.com


In [30]:
data = data.iloc[:,[0,1,2,3,4,5,6,7,8,9,10]]
data

,shieldid,day,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank
0,enilokmacy,2023-01-04,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0
1,enilokmacy,2023-01-10,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0
2,fecjavfec,2023-01-09,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0
3,fecjavfec,2023-01-20,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0
4,oreethecaw,2023-01-20,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8267454,thwijijod,2022-12-21,cdn-prod-sg.yepgarage.info,yepgarage.info,1,20,2022-12-21 05:28:15.901,2022-12-21 05:28:15.901,0,1,0.0
8267455,thwijijod,2023-01-12,www.omaemirates.com,omaemirates.com,1,1,2023-01-12 06:01:10.576,2023-01-12 06:01:10.576,0,1,0.0
8267456,volcagbuca,2023-01-03,www.copenhagen.com,copenhagen.com,2,2,2023-01-03 01:18:13.956,2023-01-03 01:18:13.983,0,1,0.0
8267457,volcagbuca,2023-01-20,www.jaiganticstudios.com,jaiganticstudios.com,1,1,2023-01-20 19:14:39.736,2023-01-20 19:14:39.736,0,1,0.0


In [31]:
dates = pd.to_datetime(data.timefirstseen)

In [32]:
dates1 = pd.to_datetime(data.timelastseen)

In [33]:
data['time_first_seen_hour'] = dates.dt.hour
data['time_last_seen_hour'] = dates1.dt.hour
data['day_of_week'] = dates.dt.dayofweek
data

,shieldid,day,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,time_first_seen_hour,time_last_seen_hour,day_of_week
0,enilokmacy,2023-01-04,1334519-19.chat.api.drift.com,drift.com,1,16,2023-01-04 00:15:40.307,2023-01-04 00:15:40.307,0,13,0.0,0,0,2
1,enilokmacy,2023-01-10,istio.api.drift.com,drift.com,1,24,2023-01-10 13:36:20.757,2023-01-10 13:36:20.757,0,19,0.0,13,13,1
2,fecjavfec,2023-01-09,1500556-6.chat.api.drift.com,drift.com,1,8,2023-01-09 15:00:47.243,2023-01-09 15:00:47.243,0,15,0.0,15,15,0
3,fecjavfec,2023-01-20,metrics.api.drift.com,drift.com,4,260,2023-01-20 10:08:52.388,2023-01-20 20:55:08.464,0,20,0.0,10,20,4
4,oreethecaw,2023-01-20,presence.api.drift.com,drift.com,15,140,2023-01-20 02:55:21.611,2023-01-20 14:29:37.833,0,20,0.0,2,14,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8267454,thwijijod,2022-12-21,cdn-prod-sg.yepgarage.info,yepgarage.info,1,20,2022-12-21 05:28:15.901,2022-12-21 05:28:15.901,0,1,0.0,5,5,2
8267455,thwijijod,2023-01-12,www.omaemirates.com,omaemirates.com,1,1,2023-01-12 06:01:10.576,2023-01-12 06:01:10.576,0,1,0.0,6,6,3
8267456,volcagbuca,2023-01-03,www.copenhagen.com,copenhagen.com,2,2,2023-01-03 01:18:13.956,2023-01-03 01:18:13.983,0,1,0.0,1,1,1
8267457,volcagbuca,2023-01-20,www.jaiganticstudios.com,jaiganticstudios.com,1,1,2023-01-20 19:14:39.736,2023-01-20 19:14:39.736,0,1,0.0,19,19,4


In [36]:
X = data.iloc[:,[4,5,9,10,11,12,13]]
y = data.iloc[:,8]
X

,num_clients,num_responses,shieldid_count,rank,time_first_seen_hour,time_last_seen_hour,day_of_week
0,1,16,13,0.0,0,0,2
1,1,24,19,0.0,13,13,1
2,1,8,15,0.0,15,15,0
3,4,260,20,0.0,10,20,4
4,15,140,20,0.0,2,14,4
...,...,...,...,...,...,...,...
8267454,1,20,1,0.0,5,5,2
8267455,1,1,1,0.0,6,6,3
8267456,2,2,1,0.0,1,1,1
8267457,1,1,1,0.0,19,19,4


In [37]:
y

0          0
1          0
2          0
3          0
4          0
          ..
8267454    0
8267455    0
8267456    0
8267457    0
8267458    0
Name: anomaly, Length: 8267459, dtype: int64

In [38]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [39]:
sum(y_train)

285

In [42]:
xgbc = XGBClassifier(eta = .1, # Default = .3
                    n_estimators = 100, # Default = 100
                    gamma = 0,
                    max_depth = 10,
                    subsample=.5,
                    reg_lambda = 0,
                    alpha = 0,
                    min_child_weight = 0,
                    scale_pos_weight = 1000)

In [43]:
xgbc.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric=None,
              feature_types=None, gamma=0, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=0, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [44]:
y_pred = xgbc.predict(X_test)

In [45]:
confusion_matrix(y_test,y_pred) 

array([[2061861,    4916],
       [     73,      15]], dtype=int64)

In [46]:
score = roc_auc_score(y_test, y_pred)
print('ROC AUC Score: ',score)

ROC AUC Score:  0.5840379813813752


In [48]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', 0.1,100),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }

    # Fit the model
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    score = roc_auc_score(y_test, y_pred)
    return score

In [49]:
study = optuna.create_study(direction='maximize')

[I 2023-04-01 08:29:59,759] A new study created in memory with name: no-name-b67ba009-793f-4c35-8055-4f8fc7ab0e87


In [50]:
study.optimize(objective, n_trials=25)

[I 2023-04-01 08:36:21,103] Trial 0 finished with value: 0.5 and parameters: {'max_depth': 9, 'learning_rate': 0.02780505793570594, 'n_estimators': 245, 'min_child_weight': 4, 'gamma': 0.0009386982530668787, 'subsample': 0.03320926848445162, 'colsample_bytree': 0.4409116689693423, 'reg_alpha': 8.567217946956052e-06, 'reg_lambda': 0.0829262691184126, 'scale_pos_weight': 1.1183847732106629}. Best is trial 0 with value: 0.5.
[I 2023-04-01 08:38:11,352] Trial 1 finished with value: 0.5 and parameters: {'max_depth': 7, 'learning_rate': 0.01745374097250074, 'n_estimators': 67, 'min_child_weight': 7, 'gamma': 6.193534248840947e-08, 'subsample': 0.09865390295627725, 'colsample_bytree': 0.14240357787204, 'reg_alpha': 0.03143639779792172, 'reg_lambda': 0.002787993105722964, 'scale_pos_weight': 56.72190141808899}. Best is trial 0 with value: 0.5.
[I 2023-04-01 08:50:13,783] Trial 2 finished with value: 0.5 and parameters: {'max_depth': 6, 'learning_rate': 0.08666320987493588, 'n_estimators': 443,

KeyboardInterrupt: 

In [ ]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

In [ ]:
params = trial.params

In [ ]:
model = XGBClassifier(max_depth = 7,
                     eta = 0.7606116184994766,
                     n_estimators = 491,
                     min_child_weight = 6,
                     gamma = 0.0013946515176432887,
                     subsample = 0.522620787079201,
                     colsample_bytree = 0.012795671270916107,
                     reg_alpha = 2.1096388009025918e-06,
                     reg_lambda= 1.1376923345741148e-06,
                     scale_pos_weight: 87.46202338328155)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
confusion_matrix(y_test,y_pred) 

# Set 4

In [2]:
data_s4 = pd.read_csv('updated_set4.csv')
data_s4

,Unnamed: 0,day,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank
0,0,2022-12-02,ajkerabo,assets.msn.com,msn.com,1,317,2022-12-02 00:00:49.999,2022-12-02 23:51:51.059,0,47,26.0
1,1,2022-11-27,ajkerabo,login.live.com,live.com,1,96,2022-11-27 06:41:01.322,2022-11-27 23:13:15.499,0,47,12.0
2,2,2022-12-05,ajkerabo,oneclient.sfx.ms,sfx.ms,1,4,2022-12-05 21:35:04.706,2022-12-05 22:05:33.587,0,50,NaN
3,3,2022-12-01,avdisabgo,a.omappapi.com,omappapi.com,1,24,2022-12-01 00:30:04.003,2022-12-01 23:30:03.839,0,18,NaN
4,4,2022-12-04,avdisabgo,a934.b.akamai.net,akamai.net,1,48,2022-12-04 00:45:05.042,2022-12-04 23:45:04.890,0,35,32355.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4728846,4728846,2022-12-02,zicudiva,trouter2-uswc-1-b.trouter.teams.microsoft.com,microsoft.com,1,20,2022-12-02 12:14:14.659,2022-12-02 20:22:01.156,0,53,46.0
4728847,4728847,2022-12-05,zicudiva,windows.msn.com,msn.com,6,16,2022-12-05 12:28:44.771,2022-12-05 23:14:29.640,0,47,26.0
4728848,4728848,2022-11-15,zicudiva,www.dutch-housing.com,dutch-housing.com,1,4,2022-11-15 22:49:32.407,2022-11-15 22:49:32.407,0,1,NaN
4728849,4728849,2022-11-30,zicudiva,www.ebay.com,ebay.com,1,1,2022-11-30 17:26:39.320,2022-11-30 17:26:39.320,0,19,19.0


In [3]:
data_s4['rank'] = data_s4['rank'].fillna(0)
data_s4

,Unnamed: 0,day,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank
0,0,2022-12-02,ajkerabo,assets.msn.com,msn.com,1,317,2022-12-02 00:00:49.999,2022-12-02 23:51:51.059,0,47,26.0
1,1,2022-11-27,ajkerabo,login.live.com,live.com,1,96,2022-11-27 06:41:01.322,2022-11-27 23:13:15.499,0,47,12.0
2,2,2022-12-05,ajkerabo,oneclient.sfx.ms,sfx.ms,1,4,2022-12-05 21:35:04.706,2022-12-05 22:05:33.587,0,50,0.0
3,3,2022-12-01,avdisabgo,a.omappapi.com,omappapi.com,1,24,2022-12-01 00:30:04.003,2022-12-01 23:30:03.839,0,18,0.0
4,4,2022-12-04,avdisabgo,a934.b.akamai.net,akamai.net,1,48,2022-12-04 00:45:05.042,2022-12-04 23:45:04.890,0,35,32355.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4728846,4728846,2022-12-02,zicudiva,trouter2-uswc-1-b.trouter.teams.microsoft.com,microsoft.com,1,20,2022-12-02 12:14:14.659,2022-12-02 20:22:01.156,0,53,46.0
4728847,4728847,2022-12-05,zicudiva,windows.msn.com,msn.com,6,16,2022-12-05 12:28:44.771,2022-12-05 23:14:29.640,0,47,26.0
4728848,4728848,2022-11-15,zicudiva,www.dutch-housing.com,dutch-housing.com,1,4,2022-11-15 22:49:32.407,2022-11-15 22:49:32.407,0,1,0.0
4728849,4728849,2022-11-30,zicudiva,www.ebay.com,ebay.com,1,1,2022-11-30 17:26:39.320,2022-11-30 17:26:39.320,0,19,19.0


In [4]:
dates_s4 = pd.to_datetime(data_s4.timefirstseen)

In [5]:
dates_s4_last = pd.to_datetime(data_s4.timelastseen)

In [6]:
data_s4['time_first_seen_hour'] = dates_s4.dt.hour
data_s4['time_last_seen_hour'] = dates_s4_last.dt.hour
data_s4['day_of_week'] = dates_s4.dt.dayofweek
data_s4

,Unnamed: 0,day,shieldid,dns_qname,dns_qdomainname,num_clients,num_responses,timefirstseen,timelastseen,anomaly,shieldid_count,rank,time_first_seen_hour,time_last_seen_hour,day_of_week
0,0,2022-12-02,ajkerabo,assets.msn.com,msn.com,1,317,2022-12-02 00:00:49.999,2022-12-02 23:51:51.059,0,47,26.0,0,23,4
1,1,2022-11-27,ajkerabo,login.live.com,live.com,1,96,2022-11-27 06:41:01.322,2022-11-27 23:13:15.499,0,47,12.0,6,23,6
2,2,2022-12-05,ajkerabo,oneclient.sfx.ms,sfx.ms,1,4,2022-12-05 21:35:04.706,2022-12-05 22:05:33.587,0,50,0.0,21,22,0
3,3,2022-12-01,avdisabgo,a.omappapi.com,omappapi.com,1,24,2022-12-01 00:30:04.003,2022-12-01 23:30:03.839,0,18,0.0,0,23,3
4,4,2022-12-04,avdisabgo,a934.b.akamai.net,akamai.net,1,48,2022-12-04 00:45:05.042,2022-12-04 23:45:04.890,0,35,32355.0,0,23,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4728846,4728846,2022-12-02,zicudiva,trouter2-uswc-1-b.trouter.teams.microsoft.com,microsoft.com,1,20,2022-12-02 12:14:14.659,2022-12-02 20:22:01.156,0,53,46.0,12,20,4
4728847,4728847,2022-12-05,zicudiva,windows.msn.com,msn.com,6,16,2022-12-05 12:28:44.771,2022-12-05 23:14:29.640,0,47,26.0,12,23,0
4728848,4728848,2022-11-15,zicudiva,www.dutch-housing.com,dutch-housing.com,1,4,2022-11-15 22:49:32.407,2022-11-15 22:49:32.407,0,1,0.0,22,22,1
4728849,4728849,2022-11-30,zicudiva,www.ebay.com,ebay.com,1,1,2022-11-30 17:26:39.320,2022-11-30 17:26:39.320,0,19,19.0,17,17,2


In [7]:
X = data_s4.iloc[:,[5,6,10,11,12,13,14]]
y = data_s4.iloc[:,9]
X

,num_clients,num_responses,shieldid_count,rank,time_first_seen_hour,time_last_seen_hour,day_of_week
0,1,317,47,26.0,0,23,4
1,1,96,47,12.0,6,23,6
2,1,4,50,0.0,21,22,0
3,1,24,18,0.0,0,23,3
4,1,48,35,32355.0,0,23,6
...,...,...,...,...,...,...,...
4728846,1,20,53,46.0,12,20,4
4728847,6,16,47,26.0,12,23,0
4728848,1,4,1,0.0,22,22,1
4728849,1,1,19,19.0,17,17,2


In [8]:
sum(y)

5978

In [9]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [10]:
xgbc = XGBClassifier(eta = .1, # Default = .3
                    n_estimators = 100, # Default = 100
                    gamma = 0,
                    max_depth = 10,
                    subsample=.5,
                    reg_lambda = 0,
                    alpha = 0,
                    min_child_weight = 0,
                    scale_pos_weight = 1000)

In [11]:
xgbc.fit(X_train, y_train)

XGBClassifier(alpha=0, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.1, eval_metric=None,
              feature_types=None, gamma=0, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=10,
              max_leaves=None, min_child_weight=0, missing=nan,
              monotone_constraints=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [12]:
y_pred = xgbc.predict(X_test)

In [13]:
confusion_matrix(y_test,y_pred) 

array([[1077917,  102807],
       [    161,    1328]], dtype=int64)

In [14]:
score = roc_auc_score(y_test, y_pred)
print('ROC AUC Score: ',score)


ROC AUC Score:  0.9024012939060015


In [19]:
def objective(trial):
    """Define the objective function"""

    params = {
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 500),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 1e-8, 1.0),
        'subsample': trial.suggest_loguniform('subsample', 0.01, 1.0),
        'colsample_bytree': trial.suggest_loguniform('colsample_bytree', 0.01, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'scale_pos_weight': trial.suggest_loguniform('scale_pos_weight', 0.1,100),
        'eval_metric': 'mlogloss',
        'use_label_encoder': False
    }

    # Fit the model
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train, y_train)

    # Make predictions
    y_pred = optuna_model.predict(X_test)

    # Evaluate predictions
    score = roc_auc_score(y_test, y_pred)
    return score

In [20]:
study = optuna.create_study(direction='maximize')


[I 2023-03-31 21:22:29,540] A new study created in memory with name: no-name-6d722b6d-cd6e-4980-b154-286643357d32


In [21]:
study.optimize(objective, n_trials=100)


[I 2023-03-31 21:23:13,485] Trial 0 finished with value: 0.5 and parameters: {'max_depth': 3, 'learning_rate': 0.0225487511234251, 'n_estimators': 88, 'min_child_weight': 6, 'gamma': 0.008162859515465741, 'subsample': 0.0182297013281029, 'colsample_bytree': 0.04591087734081249, 'reg_alpha': 1.8846371126784688e-05, 'reg_lambda': 8.489549093298325e-05, 'scale_pos_weight': 10.489215401437907}. Best is trial 0 with value: 0.5.
[I 2023-03-31 21:24:11,666] Trial 1 finished with value: 0.5 and parameters: {'max_depth': 8, 'learning_rate': 0.016554515603084862, 'n_estimators': 108, 'min_child_weight': 4, 'gamma': 0.0018665701142880485, 'subsample': 0.036069271620923035, 'colsample_bytree': 0.2101279506806252, 'reg_alpha': 0.03365396050643127, 'reg_lambda': 1.2220647376136e-06, 'scale_pos_weight': 1.2513658634822664}. Best is trial 0 with value: 0.5.
[I 2023-03-31 21:24:41,345] Trial 2 finished with value: 0.5099404823520319 and parameters: {'max_depth': 2, 'learning_rate': 0.20670688072191107,

[I 2023-03-31 22:29:53,426] Trial 18 finished with value: 0.6403135851634613 and parameters: {'max_depth': 3, 'learning_rate': 0.9818421153760591, 'n_estimators': 288, 'min_child_weight': 5, 'gamma': 0.0014392698624579605, 'subsample': 0.5996859295248698, 'colsample_bytree': 0.019986595002081647, 'reg_alpha': 2.2320569932351846e-06, 'reg_lambda': 1.2760264185090183e-07, 'scale_pos_weight': 44.1807175443522}. Best is trial 12 with value: 0.8294328212309089.
[I 2023-03-31 22:32:49,774] Trial 19 finished with value: 0.7477563324574465 and parameters: {'max_depth': 6, 'learning_rate': 0.5119975977608751, 'n_estimators': 218, 'min_child_weight': 3, 'gamma': 0.009041062598700368, 'subsample': 0.2959697825683648, 'colsample_bytree': 0.0815065733172687, 'reg_alpha': 1.4609342095391769e-06, 'reg_lambda': 0.8915924551561661, 'scale_pos_weight': 21.859718176706618}. Best is trial 12 with value: 0.8294328212309089.
[I 2023-03-31 22:36:56,746] Trial 20 finished with value: 0.5100717577389978 and pa

[I 2023-03-31 23:31:20,263] Trial 36 finished with value: 0.7070534549530662 and parameters: {'max_depth': 3, 'learning_rate': 0.7041148721322703, 'n_estimators': 409, 'min_child_weight': 2, 'gamma': 0.9502416595424988, 'subsample': 0.9998698083850369, 'colsample_bytree': 0.022661795803838106, 'reg_alpha': 1.1459517166815734e-07, 'reg_lambda': 1.8487305571556526e-06, 'scale_pos_weight': 25.96869925444324}. Best is trial 12 with value: 0.8294328212309089.
[I 2023-03-31 23:32:01,619] Trial 37 finished with value: 0.5086540438521939 and parameters: {'max_depth': 1, 'learning_rate': 0.7628714120941805, 'n_estimators': 121, 'min_child_weight': 3, 'gamma': 0.12183947229769612, 'subsample': 0.33859233531175204, 'colsample_bytree': 0.012917631421071368, 'reg_alpha': 2.551849000532525e-05, 'reg_lambda': 6.7616626275030826e-06, 'scale_pos_weight': 19.690197137876886}. Best is trial 12 with value: 0.8294328212309089.
[I 2023-03-31 23:35:30,632] Trial 38 finished with value: 0.7428922362438724 and

[I 2023-04-01 00:17:25,415] Trial 54 finished with value: 0.7983282932806826 and parameters: {'max_depth': 9, 'learning_rate': 0.5165278953171987, 'n_estimators': 105, 'min_child_weight': 2, 'gamma': 0.00017631797255378528, 'subsample': 0.2875693421619914, 'colsample_bytree': 0.02575396419559667, 'reg_alpha': 4.6543272867238304e-05, 'reg_lambda': 7.96131062967164e-08, 'scale_pos_weight': 73.16922396314197}. Best is trial 12 with value: 0.8294328212309089.
[I 2023-04-01 00:19:50,329] Trial 55 finished with value: 0.7669703335019254 and parameters: {'max_depth': 8, 'learning_rate': 0.35708125676055785, 'n_estimators': 137, 'min_child_weight': 3, 'gamma': 4.3531631860792225e-05, 'subsample': 0.4189491899442423, 'colsample_bytree': 0.021114439930577454, 'reg_alpha': 1.975798354269424e-05, 'reg_lambda': 2.3368272420752862e-08, 'scale_pos_weight': 40.6804516743183}. Best is trial 12 with value: 0.8294328212309089.
[I 2023-04-01 00:21:41,157] Trial 56 finished with value: 0.8101932613159464 a

[I 2023-04-01 01:27:05,281] Trial 72 finished with value: 0.8050727331567307 and parameters: {'max_depth': 9, 'learning_rate': 0.7148566225998659, 'n_estimators': 497, 'min_child_weight': 3, 'gamma': 2.2049025386456228e-05, 'subsample': 0.8502897359421939, 'colsample_bytree': 0.014733388102834161, 'reg_alpha': 1.9806450202191085e-05, 'reg_lambda': 1.0044915283120394e-07, 'scale_pos_weight': 48.83802479271711}. Best is trial 65 with value: 0.8306361844431297.
[I 2023-04-01 01:35:00,832] Trial 73 finished with value: 0.8242870973777711 and parameters: {'max_depth': 9, 'learning_rate': 0.569695707039744, 'n_estimators': 470, 'min_child_weight': 3, 'gamma': 9.980126006581932e-05, 'subsample': 0.8322916344356786, 'colsample_bytree': 0.011679610551727282, 'reg_alpha': 1.3336057967691172e-06, 'reg_lambda': 2.5979070449009304e-07, 'scale_pos_weight': 78.55568876098448}. Best is trial 65 with value: 0.8306361844431297.
[I 2023-04-01 01:42:51,095] Trial 74 finished with value: 0.8254630104711635

[I 2023-04-01 03:45:46,370] Trial 90 finished with value: 0.6692150164031012 and parameters: {'max_depth': 8, 'learning_rate': 0.9164815812439578, 'n_estimators': 449, 'min_child_weight': 5, 'gamma': 2.891867376493316e-05, 'subsample': 0.4955144188916397, 'colsample_bytree': 0.017519848351362676, 'reg_alpha': 6.145781488609365e-07, 'reg_lambda': 1.693184315422644e-06, 'scale_pos_weight': 67.07471978593463}. Best is trial 65 with value: 0.8306361844431297.
[I 2023-04-01 03:54:01,933] Trial 91 finished with value: 0.8299316526282724 and parameters: {'max_depth': 9, 'learning_rate': 0.6419923240010542, 'n_estimators': 485, 'min_child_weight': 5, 'gamma': 0.00034818422272491926, 'subsample': 0.7991049801294821, 'colsample_bytree': 0.012616677728965913, 'reg_alpha': 2.1427176527431345e-06, 'reg_lambda': 5.252304995182357e-07, 'scale_pos_weight': 86.16577226422633}. Best is trial 65 with value: 0.8306361844431297.
[I 2023-04-01 04:02:43,199] Trial 92 finished with value: 0.831203254640346 an

In [22]:
print('Number of finished trials: {}'.format(len(study.trials)))
print('Best trial:')
trial = study.best_trial

print('  Value: {}'.format(trial.value))
print('  Params: ')

for key, value in trial.params.items():
    print('    {}: {}'.format(key, value))

Number of finished trials: 100
Best trial:
  Value: 0.8333844378971822
  Params: 
    max_depth: 7
    learning_rate: 0.7606116184994766
    n_estimators: 491
    min_child_weight: 6
    gamma: 0.0013946515176432887
    subsample: 0.522620787079201
    colsample_bytree: 0.012795671270916107
    reg_alpha: 2.1096388009025918e-06
    reg_lambda: 1.1376923345741148e-06
    scale_pos_weight: 87.46202338328155


In [23]:
params = trial.params

In [24]:
model = XGBClassifier(max_depth = 7,
                     eta = 0.7606116184994766,
                     n_estimators = 491,
                     min_child_weight = 6,
                     gamma = 0.0013946515176432887,
                     subsample = 0.522620787079201,
                     colsample_bytree = 0.012795671270916107,
                     reg_alpha = 2.1096388009025918e-06,
                     reg_lambda= 1.1376923345741148e-06,
                     scale_pos_weight: 87.46202338328155)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.012795671270916107, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.0013946515176432887, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.7606116184994766, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=7, max_leaves=None,
              min_child_weight=6, missing=nan, monotone_constraints=None,
              n_estimators=491, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [25]:
y_pred = model.predict(X_test)

In [26]:
confusion_matrix(y_test,y_pred) 

array([[1166307,   14417],
       [    478,    1011]], dtype=int64)